In [160]:
%serialconnect


Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [253]:
%sendtofile --source ../../jupyter_micropython_developer_notebooks/Sensor_Kennel/BNO055serial_funcs.py
%sendtofile --source ../../jupyter_micropython_developer_notebooks/Sensor_Kennel/BNO055_funcs.py



Sent 43 lines (1708 bytes) to BNO055_funcs.py.


In [109]:
%serialconnect


Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [310]:
# i2c version (not working)
#from machine import I2C, Pin
#i2c = I2C(scl=Pin(15, Pin.OUT, P`in.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=450000)
#print(i2c.scan())`
#print(i2c.readfrom_mem(0x28, 0x00, 6))


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: [Errno 19] ENODEV


In [117]:
%serialconnect
from machine import UART
uart = UART(1, baudrate=115200, rx=13, tx=17)
uart.read()
#  Pin-2 TX (17), Upper side Pin-2 RX (13) 
#  For BNO055 with PS1=High SCL=TX(17), SDA=RX(13)


serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [118]:
uart.write(b"\xAA\x01\x00\x06")  # request chip_id and firmware version
time.sleep_ms(20)
r = uart.read()
print(r)


b'\xbb\x06\xa0\xfb2\x0f\x08\x03'


In [119]:
from BNO055serial_funcs import InitBNO055
r = uart.read()
print(r)


None


In [120]:
import ustruct
def InitBNO055():
    bno055write1(0x3D, 0x00)   # PWR_MODE
    bno055write1(0x3B, 0x00)   # UNIT_SEL, celsius, UDegrees and m/s^2
    bno055write1(0x3D, 0x0C)   # back to NDOF mode
    print("Temperature", bno055read(0x34, 1)[0])

def bno055write1(reg, val):
    uart.write(bytes((0xAA, 0x00, reg, 1, val)))
    time.sleep_ms(20)
    v = uart.read()
    if v != b'\xee\x01':
        raise Exception("bad bno055write %s" % str(v))

def bno055read(reg, n):
    uart.write(b"\xAA\x01")
    uart.write(chr(reg))
    uart.write(chr(n))
    time.sleep_ms(20)
    r = uart.read()
    if not ((r[0] == 0xBB) and (r[1] == n) and (len(r) == n + 2)):
        raise Exception("bad bno055read %s" % str(r))
    return r[2:]



In [121]:
InitBNO055()

Temperature 27


In [102]:
print(BNO055absorientation())

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'BNO055absorientation' is not defined


In [126]:
import array 
n = 200
a = None
a = array.array("h", range(n))
for i in range(3, -1, -1):
    time.sleep(1.0)
    print("countdown", i)
t0 = time.ticks_ms()
for i in range(n):
    uart.write(b"\xAA\x01\x28\x02")
    time.sleep_ms(10)
    r = uart.read()
    if r and (r[0] == 0xBB) and (r[1] == 2) and (len(r) == 4):
        a[i] = (r[2]+r[3]*256)
    else:
        a[i] = 32768
        print(r)
tdur = time.ticks_ms() - t0
print(tdur/1000, tdur/n)

countdown 3
countdown 2
countdown 1
countdown 0
b'\xee\x07'
b'\xee\x07'
b'\xee\x07'
b'\xee\x07'
2.06 10.3


In [127]:
print(r)

b'\xbb\x02\n\x00'


In [128]:
print(a)

array('h', [3, 4, 7, 10, 12, 12, 4, 9, 11, 9, 8, 8, 9, 6, 2, 0, -4, -4, -2, 1, 0, -1, -1, 3, 0, -1, -2, 0, -1, -1, -4, 0, 5, -32768, 9, 13, 13, 8, 2, 7, 13, 10, 11, 12, 15, 9, 6, 10, 10, 4, 6, 9, 5, 1, -1, 0, 5, 8, 7, 5, 7, 6, 0, 0, 4, 1, 1, 0, -2, -32768, -2, 0, 6, 8, 5, 1, 2, -1, 0, -1, -5, -11, -17, -35, -49, -67, -112, -257, -324, -374, -410, -490, -535, -601, -646, -715, -756, -786, -777, -699, -684, -657, -589, -438, -282, -32768, -103, -35, 35, 111, 252, 303, 378, 432, 488, 530, 529, 525, 522, 477, 467, 465, 460, 521, 560, 598, 596, 604, 595, 599, 610, 627, 639, 625, 617, 592, 582, 566, 548, 529, 525, 526, 512, 498, 468, 427, 374, 366, 359, 346, 328, 264, 226, 198, 170, 108, 68, 38, 25, -33, -64, -111, -165, -278, -323, -354, -361, -341, -339, -332, -334, -421, -461, -489, -497, -463, -440, -418, -371, -32768, -336, -315, -306, -308, -312, -294, -281, -256, -257, -242, -205, -148, -99, -50, -22, -44, -36, -14, 3, 10])


In [82]:
uart.write(b"\xAA\x01\x28\x02")
time.sleep_ms(10)
r = uart.read()
if r and (r[0] == 0xBB) and (r[1] == 2) and (len(r) == 4):
    print(r[2]+r[3]*256)

11


In [88]:
import time
calibstat = 0
for i in range(10000):
    uart.write(b"\xAA\x01\x35\x01")
    time.sleep_ms(20)
    r = uart.read()
    if r and r[0] == 0xBB and r[1] == 1 and len(r) == 3:
        lcalibstat = r[2]
    if lcalibstat != calibstat:
        calibstat = lcalibstat`
        print("sys:", (calibstat>>6)&0x03, "gyr:", (calibstat>>4)&0x03, "acc:", (calibstat>>2)&0x03, "mag:", calibstat&0x03)


sys: 3 gyr: 3 acc: 1 mag: 3
sys: 2 gyr: 3 acc: 1 mag: 3
sys: 2 gyr: 3 acc: 3 mag: 3
sys: 3 gyr: 3 acc: 3 mag: 3
.sys: 2 gyr: 3 acc: 3 mag: 3
sys: 3 gyr: 3 acc: 3 mag: 3
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 5, in <module>
KeyboardInterrupt: 


In [87]:
print(bin(calibstat))

0b11110111
